In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import pandas as pd

from brainpipe.system import study
from utils import odor_list_su, rename_elecs
from scipy.stats import ttest_ind, ttest_1samp
from mne.stats import fdr_correction, bonferroni_correction

In [3]:
"""create diff wth - btw by odor and by Memory Richness conditions"""
st = study('Olfacto')
exp = 'Enc'#'Enc'
freq = 'theta' #freqname in filename
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/')
path_save = path.join(path_pow, 'by_cont_{}_v=1_elecs=all/')
filename = path.join(path_save, 'TPS_pears_{}_odor{}_{}_{}.npz')

for su in odor_list_su(exp):
    for od in odor_list_su(exp)[su]:
        mat_wth = np.load(filename.format('wth',su,od,'wth',freq),allow_pickle=True)
        tps_wth = mat_wth['tps']
        tps_btw = np.load(filename.format('btw',su,od,'btw',freq))['tps']
        diff = tps_wth - np.mean(tps_btw,axis=1)[:,np.newaxis]
        
        dict_save = {}
        for file in mat_wth.files:
            dict_save[file] = mat_wth[file] if file != 'tps' else diff
        if not path.exists(path_save.format('diff')):
            makedirs(path_save.format('diff'))
        np.savez(filename.format('diff',su,od,'diff',freq),**dict_save)
        

-> Olfacto loaded


In [46]:
"""concat diff wth - btw into Low Mid High conditions"""
from utils import odor_groups_3wgth

st = study('Olfacto')
exp = 'Enc'#'Enc'
freq = 'theta' #freqname in filename
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/')
filename = path.join(path_pow, 'by_cont_{}_v=1_elecs=all/TPS_pears_{}_odor{}_{}_{}.npz')
savename = path.join(path_pow, 'by_cont_{}_v=1_elecs=all/TPS_pears_{}_{}_{}_{}.npz')

for su in odor_groups_3wgth:
    for cond in odor_groups_3wgth[su]:
        tps_data = np.array([])
        for od in odor_groups_3wgth[su][cond]:
            if int(od) in odor_list_su(exp)[su]:
                mat = np.load(filename.format('diff',su,od,'diff',freq),allow_pickle=True)
                tps_data = np.concatenate((tps_data, mat['tps']),axis=-1) if np.size(tps_data) else mat['tps']
                print(su, od, cond, mat['tps'].shape,tps_data.shape)
        dict_tps = {}
        for file in mat.files:
            dict_tps[file] = mat[file]
        dict_tps['xpow'] = tps_data
        np.savez(savename.format('diff',su,cond,'diff',freq),**dict_tps)

-> Olfacto loaded
CHAF 2 low (61, 15) (61, 15)
CHAF 8 mid (61, 30) (61, 30)
CHAF 3 high (61, 15) (61, 15)
CHAF 7 high (61, 36) (61, 51)
CHAF 9 high (61, 40) (61, 91)
VACJ 10 low (39, 27) (39, 27)
VACJ 11 low (39, 27) (39, 54)
VACJ 14 low (39, 21) (39, 75)
VACJ 12 mid (39, 20) (39, 20)
VACJ 17 mid (39, 16) (39, 36)
VACJ 16 high (39, 21) (39, 21)
VACJ 13 high (39, 32) (39, 53)
VACJ 15 high (39, 16) (39, 69)
SEMC 10 low (53, 140) (53, 140)
SEMC 7 low (53, 36) (53, 176)
SEMC 13 low (53, 170) (53, 346)
SEMC 11 mid (53, 92) (53, 92)
SEMC 12 mid (53, 126) (53, 218)
SEMC 5 high (53, 30) (53, 30)
SEMC 9 high (53, 30) (53, 60)
SEMC 8 high (53, 30) (53, 90)
PIRJ 9 low (18, 96) (18, 96)
PIRJ 1 low (18, 105) (18, 201)
PIRJ 18 mid (18, 85) (18, 85)
PIRJ 4 mid (18, 72) (18, 157)
LEFC 2 low (27, 50) (27, 50)
LEFC 1 low (27, 26) (27, 76)
LEFC 16 low (27, 384) (27, 460)
LEFC 15 mid (27, 319) (27, 319)
LEFC 3 mid (27, 44) (27, 363)
LEFC 4 high (27, 44) (27, 44)
LEFC 17 high (27, 231) (27, 275)
LEFC 14 hi

In [4]:
"""Ttests for significance of Wth - Btw and Diff consistancy"""
st = study('Olfacto')
exp = 'Enc'#'Enc'
exp2 = 'Enc'
freqs = ['theta'] #freqname in filename
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/')
filename = path.join(path_pow, 'by_cont_{}_v=1_elecs=all/TPS_pears_{}_odor{}_{}_{}.npz')
save_path = path.join(path_pow, 'Ttest_cont_identity_v=1_elecs=all/')
if not path.exists(save_path):
    makedirs(save_path)
df_name = path.join(save_path, '{}_Ttests_1samp_{}_{}_fsf.csv') #su, conds0, conds1, freq
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf','SFG']

for freq in freqs:
    subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
    channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps0_c, tps1_c, tps2_c = np.array([]), np.array([]), np.array([])
    T_vals_c_wth, T_vals_c_btw, T_vals_c_diff = np.array([]), np.array([]), np.array([])
    p_vals_c_wth, p_fdr_c_wth, p_bf_c_wth = np.array([]), np.array([]), np.array([])
    p_vals_c_btw, p_fdr_c_btw, p_bf_c_btw = np.array([]), np.array([]), np.array([])
    p_vals_c_diff, p_fdr_c_diff, p_bf_c_diff = np.array([]), np.array([]), np.array([])
    
    for su in odor_list_su(exp2):
        #diff tps => wth_tps - btw tps (averaged) nelecs x ncombs
        wth_tps, btw_tps, diff_tps = np.array([]), np.array([]), np.array([])
        for od in odor_list_su(exp2)[su]:
            #load within and between data by odor
            mat0 = np.load(filename.format('wth',su,od,'wth',freq),allow_pickle=True)
            print(filename.format('wth',su,od,'wth',freq),mat0.files)
            print('wth',mat0['tps'].shape)
            tps_btw = np.load(filename.format('btw',su,od,'btw',freq))['tps']
            print('btw',tps_btw.shape)
            mean_btw = np.mean(tps_btw,axis=-1)[:,np.newaxis]
            diff_ = mat0['tps'] - mean_btw
            
            #select electrodes
            labels, channels = mat0['label'], mat0['channel']
            x, y, z = mat0['xyz'][:,0], mat0['xyz'][:,1], mat0['xyz'][:,2]
            labels_new = rename_elecs(labels,x,y,z)
            idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
            nelecs = len(idx_sel)
            print(su,od,nelecs,mat0['tps'].shape,mat0['channel'].shape)
            labels, channels = labels_new[idx_sel], channels[idx_sel]
            x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]
            
            #select electrodes and concatenate data
            wth_tps = np.concatenate((wth_tps,mat0['tps'][idx_sel,:]),axis=-1) if np.size(wth_tps) else mat0['tps'][idx_sel,:]
            btw_tps = np.concatenate((btw_tps,tps_btw[idx_sel,:]),axis=-1) if np.size(btw_tps) else tps_btw[idx_sel,:]
            diff_tps = np.concatenate((diff_tps,diff_[idx_sel,:]),axis=-1) if np.size(diff_tps) else diff_[idx_sel,:]
        
        print(su,freq,'wth',wth_tps.shape,'btw',btw_tps.shape,'diff',diff_tps.shape)
        
        #compute stats for all similarities compared to 0
        wth_tps, btw_tps = wth_tps.swapaxes(0,1), btw_tps.swapaxes(0,1) #ntrials x nelecs
        diff_tps = diff_tps.swapaxes(0,1)

        Tvals_wth, unc_p_wth = ttest_1samp(wth_tps, 0.0, axis=0)
        Tvals_btw, unc_p_btw = ttest_1samp(btw_tps, 0.0, axis=0)
        Tvals_diff, unc_p_diff = ttest_1samp(diff_tps, 0.0, axis=0)
        
        #as it is a one sided one sample t-test (only interested in positive values)
        unc_p_wth, unc_p_btw, unc_p_diff = unc_p_wth/2, unc_p_btw/2, unc_p_diff/2
        _, p_fdr_wth = fdr_correction(unc_p_wth)
        _, p_fdr_btw = fdr_correction(unc_p_btw)
        _, p_fdr_diff = fdr_correction(unc_p_diff)
        _, p_bf_wth = bonferroni_correction(unc_p_wth)
        _, p_bf_btw = bonferroni_correction(unc_p_btw)
        _, p_bf_diff = bonferroni_correction(unc_p_diff)

        #Fill the csv file with elec infos and stats
        subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
        elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
        labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
        channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
        x_c = np.hstack((x_c,x)) if np.size(x_c) else x
        y_c = np.hstack((y_c,y)) if np.size(y_c) else y
        z_c = np.hstack((z_c,z)) if np.size(z_c) else z
        
        tps0_c = np.hstack((tps0_c,np.mean(wth_tps, axis=0))) if np.size(tps0_c) else np.mean(wth_tps, axis=0)
        tps1_c = np.hstack((tps1_c,np.mean(btw_tps, axis=0))) if np.size(tps1_c) else np.mean(btw_tps, axis=0)
        tps2_c = np.hstack((tps2_c,np.mean(diff_tps, axis=0))) if np.size(tps2_c) else np.mean(diff_tps, axis=0)
        
        T_vals_c_wth = np.hstack((T_vals_c_wth,Tvals_wth)) if np.size(T_vals_c_wth) else Tvals_wth
        T_vals_c_btw = np.hstack((T_vals_c_btw,Tvals_btw)) if np.size(T_vals_c_btw) else Tvals_btw
        T_vals_c_diff = np.hstack((T_vals_c_diff,Tvals_diff)) if np.size(T_vals_c_diff) else Tvals_diff
        
        p_vals_c_wth = np.hstack((p_vals_c_wth,unc_p_wth)) if np.size(p_vals_c_wth) else unc_p_wth
        p_vals_c_btw = np.hstack((p_vals_c_btw,unc_p_btw)) if np.size(p_vals_c_btw) else unc_p_btw
        p_vals_c_diff = np.hstack((p_vals_c_diff,unc_p_diff)) if np.size(p_vals_c_diff) else unc_p_diff
        
        p_fdr_c_wth = np.hstack((p_fdr_c_wth,p_fdr_wth)) if np.size(p_fdr_c_wth) else p_fdr_wth
        p_fdr_c_btw = np.hstack((p_fdr_c_btw,p_fdr_btw)) if np.size(p_fdr_c_btw) else p_fdr_btw
        p_fdr_c_diff = np.hstack((p_fdr_c_diff,p_fdr_diff)) if np.size(p_fdr_c_diff) else p_fdr_diff
        
        p_bf_c_wth = np.hstack((p_bf_c_wth,p_bf_wth)) if np.size(p_bf_c_wth) else p_bf_wth
        p_bf_c_btw = np.hstack((p_bf_c_btw,p_bf_btw)) if np.size(p_bf_c_btw) else p_bf_btw
        p_bf_c_diff = np.hstack((p_bf_c_diff,p_bf_diff)) if np.size(p_bf_c_diff) else p_bf_diff
        
    data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                z_c[:,np.newaxis],elecs_c[:,np.newaxis],
                tps0_c[:,np.newaxis], tps1_c[:,np.newaxis], tps2_c[:,np.newaxis],
                T_vals_c_wth[:,np.newaxis],p_vals_c_wth[:,np.newaxis],p_fdr_c_wth[:,np.newaxis],p_bf_c_wth[:,np.newaxis],
                T_vals_c_btw[:,np.newaxis],p_vals_c_btw[:,np.newaxis],p_fdr_c_btw[:,np.newaxis],p_bf_c_btw[:,np.newaxis],
                T_vals_c_diff[:,np.newaxis],p_vals_c_diff[:,np.newaxis],p_fdr_c_diff[:,np.newaxis], p_bf_c_diff[:,np.newaxis]),axis=1)
    df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z','elecs_num',
                'tps_wth', 'tps_btw', 'tps_diff','Tvals_wth','unc_wth','fdr_wth','bonf_wth',
                'Tvals_btw','unc_btw','fdr_btw','bonf_btw','Tvals_diff','unc_diff','fdr_diff',
                'bonf_diff'])
    print(df.shape)
    df.to_csv(df_name.format('All_subjects','by_cont_wth_btw_diff',freq),index=False)           


-> Olfacto loaded
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/TPSim_3groups_Enc/by_cont_wth_v=1_elecs=all/TPS_pears_CHAF_odor7_wth_theta.npz ['tps', 'pval', 'label', 'channel', 'xyz']
wth (61, 36)
btw (61, 32)
CHAF 7 61 (61, 36) (61,)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/TPSim_3groups_Enc/by_cont_wth_v=1_elecs=all/TPS_pears_CHAF_odor8_wth_theta.npz ['tps', 'pval', 'label', 'channel', 'xyz']
wth (61, 30)
btw (61, 24)
CHAF 8 61 (61, 30) (61,)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/TPSim_3groups_Enc/by_cont_wth_v=1_elecs=all/TPS_pears_CHAF_odor9_wth_theta.npz ['tps', 'pval', 'label', 'channel', 'xyz']
wth (61, 40)
btw (61, 40)
CHAF 9 61 (61, 40) (61,)
/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/TPSim_3groups_Enc/by_cont_wth_v=1_elecs=all/TPS_pears_CHAF_odor2_wth_theta.npz ['tps', 'pval', 'label', 'channel', 'xyz']
wth (61, 15)
btw (61, 39)
CHAF 2 61 (61, 15) (61,)
/media/karim/Datas4To/1_Analyses_Intra

In [132]:
from collections import Counter
from os import path

fold, freq = 'Enc', 'theta'
olf_regions = ['Amg','pPirT','OFC_olf']

path_file = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/Ttest_cont_identity_v=1_elecs=all/')
df_name = path.join(path_file, 'All_subjects_Ttests_1samp_by_cont_wth_btw_diff_'+freq+'_fsf.csv') #su, conds0, conds1, freq
df_save = path.join(path_file, 'Bilan_All_subjects_Ttests_1samp_{}_{}_'+freq+'.csv')
df_init = pd.read_csv(df_name)
print('Initial df shape', df_init.shape,df_init.columns)

thrs = [0.05]
corrections = ['fdr_']

for th, corr in product(thrs,corrections):
    df = df_init.loc[(df_init['tps_wth']>0) & (df_init['tps_btw']>0) & (df_init['tps_diff']>0)]
    #df_sel = df.loc[(df[corr+'wth']<th) & (df[corr+'btw']<th) & (df[corr+'diff']<th)] #& (df[corr+'btw']<th)
    #df = df_init.loc[(df_init['tps_wth']>0)]
    df_sel = df.loc[(df[corr+'wth']<th)&(df[corr+'diff']<th)]
    df_sel['sign'] = ['similar' if t > 0 else 'different' for t in df_sel['Tvals_diff']]
    print('\n stats at p < ',th, 'correction : ',corr, df_sel.shape)
    print(Counter(df_sel['labels']))
        
    rois = np.unique(df_sel['labels'])
    for roi in rois:
        if roi == 'Amg':
            roi = 'Amg_pPirT'
            df_roi = df_sel.loc[df_sel['labels'].isin(['Amg','pPirT'])]
        elif roi != 'Amg':
            df_roi = df_sel.loc[df_sel['labels']==roi]
        df_inc = df_roi.loc[df_roi['sign']=='different'].groupby(['subjects']).count()
        df_dec = df_roi.loc[df_roi['sign']=='similar'].groupby(['subjects']).count()
           
        if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_regions):
            print(roi, 'NB of subjects with SIMILAR',df_dec.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='similar']
            print('#electrodes in total >>> ',df_plot.shape[0])
            #print(df_plot)
            df_plot.to_csv(df_save.format(roi,corr+str(th)))
            df_all = df.loc[df['labels']==roi]
        if roi == 'OFC_olf':
            df_all.to_csv(df_save.format(roi,corr+str(th)).replace('.csv','_all_elecs.csv'))
            #if roi in ['Amg_pPirT','pPirT']:
                #print(df_plot)


Initial df shape (230, 22) Index(['subjects', 'labels', 'channels', 'x', 'y', 'z', 'elecs_num', 'tps_wth',
       'tps_btw', 'tps_diff', 'Tvals_wth', 'unc_wth', 'fdr_wth', 'bonf_wth',
       'Tvals_btw', 'unc_btw', 'fdr_btw', 'bonf_btw', 'Tvals_diff', 'unc_diff',
       'fdr_diff', 'bonf_diff'],
      dtype='object')

 stats at p <  0.05 correction :  fdr_ (14, 23)
Counter({'aHC': 7, 'MFG': 2, 'IFG': 1, 'OFC_olf': 1, 'ACC': 1, 'pPirT': 1, 'Ins_olf': 1})
aHC NB of subjects with SIMILAR 3  subjects
#electrodes in total >>>  7


### Early/Late effect on uniqueness

In [ ]:
st = study('Olfacto')
exp = 'Enc'#'Enc'
exp2 = 'Enc'
freqs = ['theta'] #freqname in filename
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/')
filename = path.join(path_pow, 
            'by_odor_{}_thgh_time_v=1_elecs=all/TPS_pears_learn_{}_{}_{}_{}_2gr.npz')
save_path = path.join(path_pow, 'Ttest_odor_identity_v=1_elecs=all/')
if not path.exists(save_path):
    makedirs(save_path)
df_name = path.join(save_path, '{}_Ttests_1samp_{}_{}_learn2gr.csv') #su, conds0, conds1, freq
rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf','SFG']

for freq in freqs:
    subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
    channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps0_c, tps1_c, tps2_c, tps3_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps4_c, tps5_c, T_vals_c_diff = np.array([]), np.array([]), np.array([])
    p_vals_c_diff, p_fdr_c_diff, p_bf_c_diff = np.array([]), np.array([]), np.array([])
    
    for su in odor_list_su(exp2):
        #diff tps => wth_tps - btw tps (averaged) nelecs x ncombs
        wth_tps0, btw_tps0, diff_tps0 = np.array([]), np.array([]), np.array([])
        wth_tps1, btw_tps1, diff_tps1 = np.array([]), np.array([]), np.array([])
        for od in odor_list_su(exp2)[su]:
            #load within and between data by odor
            mat0 = np.load(filename.format('wth',su,'wth',freq,od),allow_pickle=True)
            print(filename.format('wth',su,od,'wth',freq),mat0.files)
            print('wth0',mat0['tps_0'].shape, mat0['tps_1'].shape)
            wth0, wth1 = mat0['tps_0'], mat0['tps_1']
            
            if wth0.shape[0] > 0: #make sure that odors have an E and L TPSim
            
                mat1 = np.load(filename.format('btw',su,'btw',od,freq),allow_pickle=True)
                print(mat1.files)
                btw0, btw1 = mat1['tps_0'], mat1['tps_1']
                print('btw',btw0.shape, btw1.shape)
                mean_btw0 = np.mean(btw0,axis=-1)[:,np.newaxis]
                mean_btw1 = np.mean(btw1,axis=-1)[:,np.newaxis]

                diff_0 = wth0 - mean_btw0
                diff_1 = wth1 - mean_btw1

                #select electrodes
                labels, channels = mat0['label'], mat0['channel']
                x, y, z = mat0['xyz'][:,0], mat0['xyz'][:,1], mat0['xyz'][:,2]
                labels_new = rename_elecs(labels,x,y,z)
                idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
                nelecs = len(idx_sel)
                print(su,od,nelecs,mat0['tps_0'].shape,mat0['channel'].shape)
                labels, channels = labels_new[idx_sel], channels[idx_sel]
                x, y, z = x[idx_sel], y[idx_sel], z[idx_sel]

                #select electrodes and concatenate data
                wth_tps0 = np.concatenate((wth_tps0,wth0[idx_sel,:]),axis=-1) \
                                                if np.size(wth_tps0) else wth0[idx_sel,:]
                wth_tps1 = np.concatenate((wth_tps1,wth1[idx_sel,:]),axis=-1) \
                                                if np.size(wth_tps1) else wth1[idx_sel,:]
                btw_tps0 = np.concatenate((btw_tps0,btw0[idx_sel,:]),axis=-1) \
                                                if np.size(btw_tps0) else btw0[idx_sel,:]
                btw_tps1 = np.concatenate((btw_tps1,btw1[idx_sel,:]),axis=-1) \
                                                if np.size(btw_tps1) else btw1[idx_sel,:]
                diff_tps0 = np.concatenate((diff_tps0,diff_0[idx_sel,:]),axis=-1) \
                                                if np.size(diff_tps0) else diff_0[idx_sel,:]
                diff_tps1 = np.concatenate((diff_tps1,diff_1[idx_sel,:]),axis=-1) \
                                                if np.size(diff_tps1) else diff_1[idx_sel,:]

        print(su,freq,'0 wth',wth_tps0.shape,'btw',btw_tps0.shape,'diff',diff_tps0.shape)
        print(su,freq,'1 wth',wth_tps1.shape,'btw',btw_tps1.shape,'diff',diff_tps1.shape)
        
        #compute stats for all similarities compared to 0
        wth_tps0, wth_tps1 = wth_tps0.swapaxes(0,1), wth_tps1.swapaxes(0,1) #ntrials x nelecs
        btw_tps0, btw_tps1 = btw_tps0.swapaxes(0,1), btw_tps1.swapaxes(0,1) #ntrials x nelecs
        diff_tps0, diff_tps1 = diff_tps0.swapaxes(0,1), diff_tps1.swapaxes(0,1) #ntrials x nelecs
        Tvals_diff, unc_p_diff = ttest_ind(diff_tps0, diff_tps1, axis=0)
        
        #as it is a one sided one sample t-test (only interested in positive values)
        _, p_fdr_diff = fdr_correction(unc_p_diff)
        _, p_bf_diff = bonferroni_correction(unc_p_diff)
        
        #Fill the csv file with elec infos and stats
        subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
        elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
        labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
        channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
        x_c = np.hstack((x_c,x)) if np.size(x_c) else x
        y_c = np.hstack((y_c,y)) if np.size(y_c) else y
        z_c = np.hstack((z_c,z)) if np.size(z_c) else z
        
        tps0_c = np.hstack((tps0_c,np.mean(wth_tps0, axis=0))) if np.size(tps0_c) \
                                                            else np.mean(wth_tps0, axis=0)
        tps1_c = np.hstack((tps1_c,np.mean(wth_tps1, axis=0))) if np.size(tps1_c) \
                                                            else np.mean(wth_tps1, axis=0)
        tps2_c = np.hstack((tps2_c,np.mean(btw_tps0, axis=0))) if np.size(tps2_c) \
                                                            else np.mean(btw_tps0, axis=0)
        tps3_c = np.hstack((tps3_c,np.mean(btw_tps1, axis=0))) if np.size(tps3_c) \
                                                            else np.mean(btw_tps1, axis=0)
        tps4_c = np.hstack((tps4_c,np.mean(diff_tps0, axis=0))) if np.size(tps4_c) \
                                                            else np.mean(diff_tps0, axis=0)
        tps5_c = np.hstack((tps5_c,np.mean(diff_tps1, axis=0))) if np.size(tps5_c) \
                                                            else np.mean(diff_tps1, axis=0)
        
        T_vals_c_diff = np.hstack((T_vals_c_diff,Tvals_diff)) if np.size(T_vals_c_diff) else Tvals_diff
        p_vals_c_diff = np.hstack((p_vals_c_diff,unc_p_diff)) if np.size(p_vals_c_diff) else unc_p_diff
        p_fdr_c_diff = np.hstack((p_fdr_c_diff,p_fdr_diff)) if np.size(p_fdr_c_diff) else p_fdr_diff
        p_bf_c_diff = np.hstack((p_bf_c_diff,p_bf_diff)) if np.size(p_bf_c_diff) else p_bf_diff
    
    print(subjects_c.shape,labels_c.shape, channels_c.shape,tps0_c.shape)
    data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],
                channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],
                z_c[:,np.newaxis],elecs_c[:,np.newaxis],
                tps0_c[:,np.newaxis], tps1_c[:,np.newaxis], tps2_c[:,np.newaxis],
                tps3_c[:,np.newaxis], tps4_c[:,np.newaxis], tps5_c[:,np.newaxis],
                T_vals_c_diff[:,np.newaxis],p_vals_c_diff[:,np.newaxis],
                p_fdr_c_diff[:,np.newaxis], p_bf_c_diff[:,np.newaxis]),axis=1)
    df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z','elecs_num',
                'tps_wth0', 'tps_wth1', 'tps_btw0', 'tps_btw1', 'tps_diff0','tps_diff1',
                'Tvals_diff','unc_diff','fdr_diff','bonf_diff'])
    print(df.shape)
    df.to_csv(df_name.format('All_subjects','by_odor_wth_btw_diff',freq),index=False)           


In [ ]:
"""Find significant electrodes for Early Late effects among GLOBAL SIGNIF ELECS"""
from collections import Counter
from os import path

############################################################################################
fold, freq = 'Enc', 'theta'
path_file = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/Ttest_odor_identity_v=1_elecs=all/')
df_name_global = path.join(path_file, 'All_subjects_Ttests_1samp_by_odor_wth_btw_diff_'+freq+'.csv') #su, conds0, conds1, freq
df_name_learn = path.join(path_file, 'All_subjects_Ttests_1samp_by_odor_wth_btw_diff_'+freq+'_learn2gr.csv') #su, conds0, conds1, freq
df_save = path.join(path_file, 'Bilan_All_subjects_Ttests_1samp_{}_{}_'+freq+'_learn.csv')
############################################################################################
thrs = [0.05]
corrections = ['fdr'] #'bf'

for th, corr in product(thrs,corrections):
    df_init = pd.read_csv(df_name_global)
    df_learn = pd.read_csv(df_name_learn)
    df_init['learn_'+corr] = df_learn['unc_diff']
    print('Initial df shape', df_init.shape,df_init.columns)
    print('Learn df shape', df_learn.shape,df_learn.columns)

    #select significant electrodes for GLOBAL
    df = df_init.loc[(df_init['tps_wth']>0) & (df_init['tps_btw']>0) & (df_init['tps_diff']>0)]
    df_sel = df.loc[(df[corr+'_wth']<th) & (df[corr+'_btw']<th) & (df[corr+'_diff']<th)\
                   & (df_init['learn_'+corr]<th)] #& (df[corr+'btw']<th)
    
    #df_sel = df.loc[(df[corr+'btw']<th)]
    df_sel['sign'] = ['similar' if t > 0 else 'different' for t in df_sel['Tvals_diff']]
    print('\n stats at p < ',th, 'correction : ',corr, df_sel.shape)
    print(Counter(df_sel['labels']))
    print(df_sel)
        
    rois = np.unique(df_sel['labels'])
    for roi in rois:
        if roi == 'Amg':
            roi = 'Amg_pPirT'
            df_roi = df_sel.loc[df_sel['labels'].isin(['Amg','pPirT'])]
        elif roi != 'Amg':
            df_roi = df_sel.loc[df_sel['labels']==roi]
        df_inc = df_roi.loc[df_roi['sign']=='different'].groupby(['subjects']).count()
        df_dec = df_roi.loc[df_roi['sign']=='similar'].groupby(['subjects']).count()

        if (df_inc.shape[0] >= 3) :#or (df_inc.shape[0] >=2 and roi in olf_regions):
            print(roi, 'NB of subjects with DIFFERENT',df_inc.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='different']
            print('#electrodes in total >>> ',df_plot.shape[0])
            #df_plot.to_csv(df_save.format(roi,corr+str(th)))
            #print(df_plot)
            
        if (df_dec.shape[0] >= 3) :#or (df_dec.shape[0] >=1 and roi in olf_regions):
            print(roi, 'NB of subjects with SIMILAR',df_dec.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='similar']
            print('#electrodes in total >>> ',df_plot.shape[0])
            print(df_plot)
            
            #df_plot.to_csv(df_save.format(roi,corr+str(th)))
            #if roi in ['Amg','pPirT']:
                #print(df_plot)


In [176]:
"""
Ttests 2-samples comparing Low and High (Wth-btw)
ONLY for ELECS signif in the first analysis (Wth Btw and Wth-Btw)
Load Mid but select Low and High for stats
"""

import statsmodels.api as sm
import statsmodels.formula.api as smf
from os import path
import pingouin as pg
from utils import odor_groups_3wgth
from scipy import stats
import math

############################################################################################
st = study('Olfacto')
fold, freq = 'Enc', 'theta'
path_df = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/Ttest_odor_identity_v=1_elecs=all/')
path_data = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/by_odor_{}_v=1_elecs=all/')
filename = path.join(path_data, 'TPS_pears_{}_{}_diff_theta.npz')
############################################################################################
corr = 'fdr'
conds = ['low','mid','high']
files = st.search('Bilan_All_subjects_Ttests_1samp',
              folder='feature/TPSim_3groups_'+fold+'/Ttest_odor_identity_v=1_elecs=all/')
dict_score = {'low':0, 'mid':1, 'high':2}
features = ['diff_low','diff_mid','diff_high']

for fi in files:
    print(fi)
    if corr in fi.split('_'):
        roi = fi.split('_')[5]
        roi = 'OFC_olf' if roi == 'OFC' else roi
        #if roi in ['OFC','aHC','Amg']:
        print('>>>> processing ',fi, roi)
        df = pd.read_csv(path_df+fi)
        _, idx = np.unique(df['subjects'], return_index=True)
        subjects = df['subjects'][np.sort(idx)]
        print('>>>> {} subjects significant in 1st analysis'.format(len(subjects)))
        print(np.unique(subjects))
        pvals, Tvals, subj, chans = [], [], [], []
        tps_low, tps_high, sem_low, sem_high = [], [], [], []
        tps_mid, sem_mid = [], []
        for su in subjects:
            #print(su)
            df_su = df.loc[df['subjects']==su]
            chans_df = df_su['channels'].values

            all_su_tps = []
            for cond in conds:
                mat = np.load(filename.format('diff',su,cond),allow_pickle=True)
                idx = [i for i,chan in enumerate(mat['channel']) if chan in chans_df]
                
                #To take all electrodes within an ROI
                #if roi != 'Amg':
                    #idx = [i for i,lab in enumerate(mat['label']) if lab == roi]
                #if roi == 'Amg':
                    #idx = [i for i,lab in enumerate(mat['label']) if lab in ['Amg','pPirT']]
                channels = mat['channel'][idx]
                data = mat['tps'][idx,:]
                all_su_tps.append(data)
                
            if all_su_tps[-1].shape[-1] > 1: #pass subjects without enough trials
                for elec in range(len(idx)):
                    df_ = pd.DataFrame()
                    df_['tps'] = np.concatenate([x[elec] for x in all_su_tps])
                    df_['mem_score'] = np.concatenate([[cond]*x.shape[1] for cond,x in zip(conds,all_su_tps)])
                    df_['mem_val'] = df_['mem_score'].map(dict_score)
                    df_ = df_.loc[df_['mem_score']!='mid']

                    ttests = pg.pairwise_gameshowell(data=df_, dv='tps',
                                                    tail='one-sided',between='mem_score') 
                    stats_ = np.round(ttests['T'].values,3),np.round(ttests['pval'].values,3)
                    pvals.append(stats_[1]), Tvals.append(stats_[0])
                    subj.append(su), chans.append(channels[elec])
                    tps_low.append(np.mean(all_su_tps[0][elec]))
                    tps_mid.append(np.mean(all_su_tps[1][elec]))
                    tps_high.append(np.mean(all_su_tps[2][elec]))
                    sem_low.append(stats.sem(all_su_tps[0][elec]))
                    sem_mid.append(stats.sem(all_su_tps[1][elec]))
                    sem_high.append(stats.sem(all_su_tps[2][elec]))
            
            else :#VACJ for aHC
                print('not enough trials',su)
                #print('low ',su,roi,np.mean(all_su_tps[0]))
                #print('high ',su,roi,np.mean(all_su_tps[2]))
                #print('sem low ',su,roi,stats.sem(all_su_tps[0]))
                #print('sem high ',su,roi,stats.sem(all_su_tps[2]))
                    
        pvals = [p if not math.isnan(p) else 1 for p in pvals]
        _, pvals_fdr = fdr_correction(pvals)
        _, pvals_bonf = bonferroni_correction(pvals)
        
        subj, chans = np.array(subj),np.array(chans)
        tps_low, tps_high, Tvals = np.array(tps_low),np.array(tps_high), np.array(Tvals)
        sem_low, sem_high, pvals = np.array(sem_low),np.array(sem_high),np.array(pvals)
        sem_mid, tps_mid = np.array(sem_mid),np.array(tps_mid)
        
        data_final = np.concatenate([subj[:,np.newaxis],chans[:,np.newaxis],
                tps_low[:,np.newaxis], sem_low[:,np.newaxis],
                tps_mid[:,np.newaxis], sem_mid[:,np.newaxis],
                tps_high[:,np.newaxis],sem_high[:,np.newaxis],
                Tvals,pvals,pvals_fdr,pvals_bonf],axis=1)
        
        df = pd.DataFrame(data_final, columns=['subjects','channels',
                    'tps_low', 'sem_low', 'tps_mid', 'sem_mid','tps_high', 'sem_high', 
                    'Tvals_diff','unc_diff','fdr_diff','bonf_diff'])
        df['fdr_diff'] = df['fdr_diff'].astype(float)
        df['Tvals_diff'] = df['Tvals_diff'].astype(float)
        df_sig = df.loc[(df['fdr_diff']<0.05)&(df['Tvals_diff']>0)]
        df_all = df.loc[(df['Tvals_diff']>0)]
        su_all, su_sig = np.unique(df_all['subjects']), np.unique(df_sig['subjects'])
        print('nb of subjects sig Low High', df_sig.groupby(['subjects']).count().shape[0])
        print(roi, su_all, su_sig)
        if set(su_all) == set(su_sig):
            df_sig.to_csv(path_df+fi.replace('1samp','Low_High_final_sig_elecs'))        
        else:
            df_sig.to_csv(path_df+fi.replace('1samp','Low_High_final_sig_elecs'))        
            missing_su = [su for su in su_all if su not in su_sig]
            df_not_sig = df_all.loc[df_all['subjects'].isin(missing_su)]
            df_not_sig.to_csv(path_df+fi.replace('1samp','Low_High_not_sig_elecs'))

-> Olfacto loaded
Bilan_All_subjects_Ttests_1samp_aHC_fdr_0.05_theta.csv
>>>> processing  Bilan_All_subjects_Ttests_1samp_aHC_fdr_0.05_theta.csv aHC
>>>> 5 subjects significant in 1st analysis
['FERJ' 'LEFC' 'PIRJ' 'SEMC' 'VACJ']
not enough trials VACJ
nb of subjects sig Low High 4
aHC ['FERJ' 'LEFC' 'PIRJ' 'SEMC'] ['FERJ' 'LEFC' 'PIRJ' 'SEMC']
Bilan_All_subjects_Ttests_1samp_Amg_pPirT_fdr_0.05_theta.csv
>>>> processing  Bilan_All_subjects_Ttests_1samp_Amg_pPirT_fdr_0.05_theta.csv Amg
>>>> 3 subjects significant in 1st analysis
['FERJ' 'LEFC' 'VACJ']
not enough trials VACJ
nb of subjects sig Low High 0
Amg ['FERJ' 'LEFC'] []
Bilan_All_subjects_Ttests_1samp_IFG_fdr_0.05_theta.csv
>>>> processing  Bilan_All_subjects_Ttests_1samp_IFG_fdr_0.05_theta.csv IFG
>>>> 3 subjects significant in 1st analysis
['FERJ' 'SEMC' 'VACJ']
not enough trials VACJ
nb of subjects sig Low High 1
IFG ['FERJ' 'SEMC'] ['SEMC']
Bilan_All_subjects_Ttests_1samp_MFG_fdr_0.05_theta.csv
>>>> processing  Bilan_All_subje

In [144]:
"""
Ttests 2-samples comparing Low and High (Wth-btw)
FOR ALL SUBJECTS WITH ELECTRODES IN ROIS
"""

import statsmodels.api as sm
import statsmodels.formula.api as smf
from os import path
import pingouin as pg
from utils import odor_groups_3wgth
from scipy import stats
import math

############################################################################################
st = study('Olfacto')
fold, freq = 'Enc', 'theta'
path_df = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/Ttest_cont_identity_v=1_elecs=all/')
path_data = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/by_cont_{}_v=1_elecs=all/')
filename = path.join(path_data, 'TPS_pears_{}_{}_diff_theta.npz')
df_save = path.join(path_df, 'Bilan_All_subjects_Ttests_{}_{}_{}.csv')
############################################################################################
corr, rois = 'fdr', ['aHC','OFC_olf']
conds = ['low','mid','high']
features = ['diff_low','diff_mid','diff_high']

for roi in rois:
    subj, chans = [], []
    tps_low, tps_high, sem_low, sem_high = [], [], [], []
    tps_mid, sem_mid = [], []
    for su in odor_groups_3wgth:
        all_su_tps = []
        for cond in conds:
            mat = np.load(filename.format('diff',su,cond),allow_pickle=True)
            idx = [i for i,lab in enumerate(mat['label']) if lab == roi]
            channels = mat['channel'][idx]
            data = mat['tps'][idx,:]
            all_su_tps.append(data)

        if len(idx) > 0: #pass subjects without enough trials
            for elec in range(len(idx)):
                subj.append(su), chans.append(channels[elec])
                tps_low.append(np.mean(all_su_tps[0][elec]))
                tps_mid.append(np.mean(all_su_tps[1][elec]))
                tps_high.append(np.mean(all_su_tps[2][elec]))
                sem_low.append(stats.sem(all_su_tps[0][elec]))
                sem_mid.append(stats.sem(all_su_tps[1][elec]))
                sem_high.append(stats.sem(all_su_tps[2][elec]))
        else:
            print('not enough trials', su, roi, [x.shape for x in all_su_tps])

    subj, chans = np.array(subj),np.array(chans)
    tps_low, tps_high = np.array(tps_low),np.array(tps_high)
    sem_low, sem_high = np.array(sem_low),np.array(sem_high)
    sem_mid, tps_mid = np.array(sem_mid),np.array(tps_mid)

    data_final = np.concatenate([subj[:,np.newaxis],chans[:,np.newaxis],
            tps_low[:,np.newaxis], sem_low[:,np.newaxis],
            tps_mid[:,np.newaxis], sem_mid[:,np.newaxis],
            tps_high[:,np.newaxis],sem_high[:,np.newaxis]],axis=1)

    df = pd.DataFrame(data_final, columns=['subjects','channels',
                'tps_low', 'sem_low', 'tps_mid', 'sem_mid','tps_high', 'sem_high'])

    df.to_csv(df_save.format('Low_High_all_not_sig',roi,freq))

-> Olfacto loaded
not enough trials CHAF aHC [(0, 3), (0, 3), (0, 10)]
not enough trials FERJ OFC_olf [(0, 6), (0, 3), (0, 6)]


/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/karim/anaconda3/envs/mne_coreg/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### Plot results by subject for NOT SIG elecs or SIG

In [174]:
from brainpipe.system import study
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np

st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_3groups_{}/Ttest_odor_identity_v=1_elecs=all/')
#file_name = join(PATH, 'Bilan_All_subjects_LinReg_all_roi_3gr_{}_fdr_0.05_theta.csv')
file_name = join(PATH, 'Bilan_All_subjects_Ttests_Low_High_not_sig_elecs_{}_fdr_0.05_theta.csv')
SAVE_PATH = join(PATH, 'Ind_elecs_{}_{}_avg_subj_Ttests_not_sig.png')
meth, exp, freq, rois = 'btw', 'Enc', 'theta', ['IFG','Amg_pPirT']
features = ['tps_low','tps_high']
#features = ['tps_wth','tps_btw','tps_diff']

for roi in rois:
    df = pd.read_csv(file_name.format(exp,roi))
    df_m = df.groupby(['subjects']).agg(['mean','sem'])
    nelecs = df_m.shape[0]

    fig, axs = plt.subplots(2,4, figsize=(15, 6), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()

    for i in range(nelecs):
        df_sel = df_m.iloc[i,:].unstack().T
        #print(df_sel)

        su =  df_m.index[i]
        data = df_sel[features].iloc[0,:].values
#         yerr = df_sd.iloc[i,:][features].values
        yerr = df_sel[features].iloc[1,:].values
        print(roi, su, data,yerr)

        xticks, w = np.arange((len(features))), 0.8
        axs[i].set_xticks(xticks,features)
        axs[i].bar(xticks,data,yerr=yerr,color='blue')
        axs[i].set_title('{} in {}'.format(su,roi))
    
    plt.setp(axs, ylim=(0,1.)) if roi == 'aHC' else plt.setp(axs, ylim=(0,1.3))
    plt.savefig(SAVE_PATH.format(exp,'diff_wth',roi))
    plt.savefig(SAVE_PATH.format(exp,'diff_wth',roi).replace('.png','.pdf'))
    plt.clf()
    plt.close()
    

-> Olfacto loaded
IFG FERJ [0.47251254 0.50797095] [nan nan]
Amg_pPirT FERJ [0.3457534  0.58904411] [nan nan]
Amg_pPirT LEFC [0.19044021 0.39859797] [nan nan]


### Plot elec number by patient

In [106]:
from brainpipe.system import study
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np

st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_3groups_{}/Ttest_cont_identity_v=1_elecs=all/')
SAVE_PATH = join(PATH, 'Nb_elecs_{}_{}_by_patient_sig.png')
meth, exp, freq = 'wth_diff', 'Enc', 'theta'
files = st.search('Bilan_wth_Ttests_1samp', folder=PATH.format(exp))

for fi in files:
    roi = fi.split('_')[-4]
    roi = 'OFC_'+roi if roi =='olf' else roi
    
    df_roi = pd.read_csv(join(PATH.format(exp)+fi))
    df_su = df_roi.groupby(['subjects']).count()
    
    #Plot nb of significant electrodes by win and subjects
    colors = {'CHAF':'darkblue', 'FERJ':'royalblue', 'LEFC':'deepskyblue', 
            'PIRJ':'yellow','SEMC':'darkorange', 'VACJ':'red'}
    
    xticks, w = np.arange(0,1), 0.5
    fig = plt.figure()
    bottom = np.zeros(1)
    for i,su in enumerate(df_su.index):
        count = df_su.iloc[i,0]
        print(su, count)
        plt.bar(1, count, color=colors[su], label=su, bottom=bottom)
        bottom += count
    plt.title('Significant electrodes by patient in '+roi)
    plt.ylabel('Nb of electrodes')
    plt.xticks(xticks,freq)
    plt.legend(loc=0,handletextpad=0.1, frameon=False)
    plt.savefig(SAVE_PATH.format(exp,meth,roi))
    plt.savefig(SAVE_PATH.format(exp,meth,roi).replace('.png','.pdf'))
    plt.clf()
    plt.close()

-> Olfacto loaded
FERJ 4
LEFC 2
VACJ 1
LEFC 1
SEMC 4
VACJ 5
LEFC 1
SEMC 5
VACJ 3
SEMC 1
VACJ 1
FERJ 1


### Plot average TPS (wth and wth-btw)

In [175]:
from brainpipe.system import study
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import numpy as np

st = study('Olfacto')
PATH = join(st.path, 'feature/TPSim_3groups_{}/Ttest_odor_identity_v=1_elecs=all/')
by_subj = False
if by_subj == True:
    SAVE_PATH = join(PATH, 'TPS_avg_su={}_roi={}_meth={}_freq={}.png')
else:
    SAVE_PATH = join(PATH, 'TPS_avg_all_su_roi={}_meth={}_freq={}.png')
meth, exp, freq = 'wth_diff', 'Enc', 'theta'
#features = ['subjects','tps_wth','tps_btw','tps_diff']
features = ['subjects','tps_low','tps_high']

#files = st.search('Bilan_wth_Ttests_1samp', folder=PATH.format(exp))
files = st.search('Bilan_All_subjects_Ttests_Low_High_not_sig_elecs_Amg', folder=PATH.format(exp))

for fi in files:
    roi = fi.split('_')[-4]
    roi = 'OFC_'+roi if roi =='olf' else roi
    
    df_roi = pd.read_csv(join(PATH.format(exp)+fi))[features]
    
    if by_subj == True:
        means = df_roi.groupby(['subjects']).mean()
        stds = df_roi.groupby(['subjects']).sem()
        for i,su in enumerate(means.index):
            mean, std = means.iloc[i], stds.iloc[i]
            xticks, w = np.arange(0,len(rois)), 0.8
            fig = plt.figure()
            plt.bar(np.arange(2), mean, yerr=std, color='orange', width=w)
            plt.title('Average TPS for all sig electrodes in '+roi)
            plt.ylabel('Mean TPS')
            plt.xticks(xticks,rois)
            plt.ylim(0,1)
            plt.savefig(SAVE_PATH.format(exp,su,roi,meth,freq))
            plt.savefig(SAVE_PATH.format(exp,su,roi,meth,freq).replace('png','pdf'))
            plt.clf()
            plt.close()
    
    elif by_subj == False:
        mean, std = df_roi.mean(), df_roi.sem()
        xticks, w = np.arange(0,len(features)-1), 0.8
        fig = plt.figure()
        plt.bar(np.arange(len(features)-1), mean, yerr=std, color='orange', width=w)
        plt.title('Average TPS for all sig electrodes in '+roi)
        plt.ylabel('Mean TPS')
        plt.xticks(xticks,[feat for feat in features if feat != 'subjects'])
        plt.ylim(0,1)
        plt.savefig(SAVE_PATH.format(exp,roi,meth,freq))
        plt.savefig(SAVE_PATH.format(exp,roi,meth,freq).replace('png','pdf'))
        plt.clf()
        plt.close()

-> Olfacto loaded


In [117]:
"""
Linear Regression of uniqueness with memory richness
Only subjects and ROIs with significant results in Wth > Btw 
Take ALL electrodes within ROI and FDR correct across subjects
"""

import statsmodels.api as sm
import statsmodels.formula.api as smf
from os import path
import pingouin as pg
from utils import odor_groups_3wgth
from scipy import stats
import math

############################################################################################
st = study('Olfacto')
fold, freq = 'Enc', 'theta'
path_df = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/Ttest_odor_identity_v=1_elecs=all/')
path_data = path.join(st.path, 'feature/TPSim_3groups_'+fold+'/by_odor_{}_v=1_elecs=all/')
filename = path.join(path_data, 'TPS_pears_{}_{}_diff_theta.npz')
############################################################################################
corr = 'fdr'
conds = ['low','mid','high']
files = st.search('Bilan_All_subjects_Ttests_1samp',
              folder='feature/TPSim_3groups_'+fold+'/Ttest_odor_identity_v=1_elecs=all/')
dict_score = {'low':0, 'mid':1, 'high':2}
features = ['diff_low','diff_mid','diff_high']

for fi in files:
    if corr in fi.split('_'):
        roi = fi.split('_')[5]
        roi = 'OFC_olf' if roi == 'OFC' else roi
        #if roi in ['OFC','aHC','Amg']:
        print('>>>> processing ',fi, roi)
        df = pd.read_csv(path_df+fi)
        _, idx = np.unique(df['subjects'], return_index=True)
        subjects = df['subjects'][np.sort(idx)]
        print('>>>> {} subjects significant'.format(len(subjects)))
        print(subjects)
        pvals, Tvals, subj, chans = [], [], [], []
        tps_low, tps_high, sem_low, sem_high = [], [], [], []
        tps_mid, sem_mid = [], []
        for su in subjects:
            df_su = df.loc[df['subjects']==su]
            #idx = df_su['elecs_num']
            all_su_tps, all_scores = [], []
            for cond in conds:
                mat = np.load(filename.format('diff',su,cond),allow_pickle=True)
                if roi != 'Amg':
                    idx_elecs = [i for i,lab in enumerate(mat['label']) if lab == roi]
                if roi == 'Amg':
                    idx_elecs = [i for i,lab in enumerate(mat['label']) if lab in ['Amg','pPirT']]
                channels = mat['channel'][idx_elecs]
                data = mat['tps'][idx_elecs,:]
                nelecs, ntrials = data.shape
                all_su_tps.append(data)
                score_ = np.array([dict_score[cond]]*ntrials)
                all_scores.append(score_)
            
            #print([x.shape for x in all_su_tps])
            if all_su_tps[-1].shape[-1] > 1: #pass subjects without enough trials
                all_tps_concat = np.concatenate(all_su_tps,axis=1)
                all_scores_concat = np.concatenate(all_scores,axis=0)
                T, unc_p = [], []
                for elec in range(nelecs):
                    Y = np.array(all_tps_concat[elec])
                    X = sm.add_constant(np.array(all_scores_concat))
                    model_ols = sm.OLS(Y,X).fit()
                    Tval, pval = np.round(model_ols.tvalues[1],3),model_ols.pvalues[1]
                    Tvals.append(Tval), pvals.append(pval)
    
                    subj.append(su), chans.append(channels[elec])
                    tps_low.append(np.mean(all_su_tps[0][elec]))
                    tps_mid.append(np.mean(all_su_tps[1][elec]))
                    tps_high.append(np.mean(all_su_tps[2][elec]))
                    sem_low.append(stats.sem(all_su_tps[0][elec]))
                    sem_mid.append(stats.sem(all_su_tps[1][elec]))
                    sem_high.append(stats.sem(all_su_tps[2][elec]))

        _, pvals_fdr = fdr_correction(pvals)
        _, pvals_bonf = bonferroni_correction(pvals)
        
        subj, chans = np.array(subj),np.array(chans)
        tps_low, tps_high, Tvals = np.array(tps_low),np.array(tps_high), np.array(Tvals)
        sem_low, sem_high, pvals = np.array(sem_low),np.array(sem_high),np.array(pvals)
        sem_mid, tps_mid = np.array(sem_mid),np.array(tps_mid)
        #print(pvals.shape, pvals_fdr.shape,pvals_bonf.shape, Tvals.shape)
        data_final = np.concatenate([subj[:,np.newaxis],chans[:,np.newaxis],
                tps_low[:,np.newaxis], sem_low[:,np.newaxis],
                tps_mid[:,np.newaxis], sem_mid[:,np.newaxis],
                tps_high[:,np.newaxis],sem_high[:,np.newaxis],
                Tvals[:,np.newaxis],pvals[:,np.newaxis],
                pvals_fdr[:,np.newaxis],pvals_bonf[:,np.newaxis]],axis=1)
        
        df = pd.DataFrame(data_final, columns=['subjects','channels',
                    'tps_low', 'sem_low', 'tps_mid', 'sem_mid','tps_high', 'sem_high', 
                    'Tvals_diff','unc_diff','fdr_diff','bonf_diff'])
        
        df['fdr_diff'] = df['fdr_diff'].astype(float)
        df['Tvals_diff'] = df['Tvals_diff'].astype(float)
        df_sig = df.loc[(df['fdr_diff']<0.05)&(df['Tvals_diff']>0)]
        nb_su = df_sig.groupby(['subjects']).count().shape[0]
        if nb_su >= 2:
            print('INCREASE' ,roi, nb_su, 'sig')
            df_sig.to_csv(path_df+fi.replace('Ttests_1samp','LinReg_all_roi_3gr'))            
            df.to_csv(path_df+fi.replace('Ttests_1samp','LinReg_all_roi_3gr_all_eles'))

-> Olfacto loaded
>>>> processing  Bilan_All_subjects_Ttests_1samp_aHC_fdr_0.05_theta.csv aHC
>>>> 5 subjects significant
0     LEFC
7     PIRJ
11    VACJ
17    SEMC
20    FERJ
Name: subjects, dtype: object
INCREASE aHC 4 sig
>>>> processing  Bilan_All_subjects_Ttests_1samp_Amg_pPirT_fdr_0.05_theta.csv Amg
>>>> 3 subjects significant
0    LEFC
1    VACJ
3    FERJ
Name: subjects, dtype: object
>>>> processing  Bilan_All_subjects_Ttests_1samp_IFG_fdr_0.05_theta.csv IFG
>>>> 3 subjects significant
0     VACJ
7     SEMC
15    FERJ
Name: subjects, dtype: object
>>>> processing  Bilan_All_subjects_Ttests_1samp_MFG_fdr_0.05_theta.csv MFG
>>>> 4 subjects significant
0    CHAF
4    LEFC
5    VACJ
9    SEMC
Name: subjects, dtype: object
>>>> processing  Bilan_All_subjects_Ttests_1samp_OFC_olf_fdr_0.05_theta.csv OFC_olf
>>>> 4 subjects significant
0    LEFC
4    PIRJ
7    VACJ
8    SEMC
Name: subjects, dtype: object
INCREASE OFC_olf 2 sig


In [ ]:
"""
ODOR 
____________________________________
/// AT ENCODING
wth > 0 btw > 0 AND wth > btw 
0.05 (Amg/pPirT,aHC,OFC)
0.01 (OFC,Amg/PirT)

/// AT RETRIEVAL
0.05 (Amg/Pir, IFG, Ins, OFC,SFG, aHC)
0.01 (Amg/Pir,IFG,SFG,aHC,Ins)

/// REINSTATEMENT (even in FDR)
nothing sig (distinct representations btw E and R even with the same odor)
Wth > 0 (Amg,IFG,OFC)
Btw > 0 (IFG, OFC and MFG representation different)
Wth > Btw (SFG, IFG)
"""

"""
CONTEXT
__________________________________
nothing in Bonferoni 
fdr 0.05 retrieval OFC (2 patients)
"""

"""
PLACE 
__________________________________
close/far (3 vs 4) nothing
close/far (2 vs 2)
at encoding OFC (and at R and ER OFC but one person)
"""

In [17]:
"""Sum-up stats for the results part"""

st = study('Olfacto')
exp = 'Enc'#'Enc'
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/')
path_df = path.join(path_pow, 'Ttest_odor_identity_v=1_elecs=all/')
df_name = path.join(path_df, 'Bilan_All_subjects_Ttests_1samp_{}_fdr_0.05_theta.csv') #su, conds0, conds1, freq

rois = ['aHC','OFC_olf','Amg_pPirT','IFG','MFG']
for roi in rois:
    df = pd.read_csv(df_name.format(roi))
    T_m = df[['Tvals_wth','Tvals_diff']].values.mean()    
    T_sd = df[['Tvals_wth','Tvals_diff']].values.std()
    p_max = df[['fdr_wth','fdr_diff']].values.max()
    print(roi, T_m, T_sd, p_max)
    
    df_su = df[['subjects','Tvals_wth','Tvals_diff','fdr_wth',
                        'fdr_diff']].groupby(['subjects']).agg(['count','mean','std','max'])
    df_su.to_csv(df_name.format(roi).replace('.csv','stat_sum.csv'))

-> Olfacto loaded
aHC 4.548683205170504 1.8991468572266996 0.011663920312600549
OFC_olf 5.958577811306919 2.2274736085545648 0.013824906833690344
Amg_pPirT 5.159190668511457 1.538865214997055 0.020837305251126474
IFG 4.46354983458307 1.5579412088879216 0.01637591659046668
MFG 4.2313884026634225 1.2148659040910557 0.017312614111290387


In [9]:
"""Sum-up stats for the results part"""

st = study('Olfacto')
exp = 'Enc'#'Enc'
path_pow = path.join(st.path, 'feature/TPSim_3groups_'+exp+'/')
path_df = path.join(path_pow, 'Ttest_odor_identity_v=1_elecs=all/')
df_name = path.join(path_df, 'Bilan_All_subjects_Ttests_Low_High_final_sig_elecs_{}_fdr_0.05_theta.csv') #su, conds0, conds1, freq

rois = ['aHC']#,'OFC_olf']#,'Amg_pPirT','IFG','MFG']
for roi in rois:
    df = pd.read_csv(df_name.format(roi))
    T_m = df[['Tvals_diff']].values.mean()    
    T_sd = df[['Tvals_diff']].values.std()
    p_max = df[['fdr_diff']].values.max()
    print(roi, T_m, T_sd, p_max)

-> Olfacto loaded
aHC 2.9598333333333335 0.7205184283247414 0.03666666666666667
OFC_olf 3.7718000000000003 1.411374280621551 0.045
